In [78]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import torch
import torch.nn as nn
from scem import kernel

In [21]:
torch.set_printoptions(precision=10)

In [146]:
bk = kernel.BKGauss(torch.tensor(1.0))
k = kernel.KGauss(torch.tensor(1.0))

bk = kernel.BKIMQ()
k = kernel.KIMQ()

In [147]:
n = 3
d = 4
dtype = torch.float64
X = torch.randn([n, d], dtype=dtype)
Y = torch.randn([n-1, d], dtype=dtype)

In [150]:
G = bk.gradX(X, Y)

In [151]:
G_ = torch.stack([k.parX(X, Y, j) for j in range(d)]).permute(1, 2, 0)

In [152]:
torch.abs(G - G_).max()

tensor(0., dtype=torch.float64)

In [153]:
G_sum_k = k.gradXY_sum(X, Y)

In [158]:
gradxy_bk = bk.gradXY(X, Y)
idx = torch.arange(d)
G_sum_bk = gradxy_bk[:, :, idx, idx].sum(axis=-1)

In [159]:
(torch.abs(G_sum_k - G_sum_bk)).max()

tensor(2.7755575616e-17, dtype=torch.float64)

In [163]:
print(G_sum_bk)

tensor([[0.0624549242, 0.0643736823],
        [0.0573358432, 0.0239614653],
        [0.0941736039, 0.0314400372]], dtype=torch.float64)


In [164]:
print(G_sum_k-G_sum_bk)

tensor([[-6.9388939039e-18, -1.3877787808e-17],
        [ 2.7755575616e-17,  0.0000000000e+00],
        [ 1.3877787808e-17, -6.9388939039e-18]], dtype=torch.float64)


In [165]:
class Net(nn.Module):
    def __init__(self, Dx):
        super(Net, self).__init__()
        self.scales = nn.Parameter(torch.ones([d]))
        
    def forward(self, X):
        return X * (self.scales**2)**0.5

In [166]:
f = Net(d)
sigma2 = 10.0
f.scales = nn.Parameter(torch.ones([d], dtype=dtype)/(sigma2)**0.5)

In [174]:
k_ = kernel.KSTFuncCompose(bk, kernel.FuncFeatureMap(f, (d, ), (d, )))

In [182]:
#k = kernel.KGauss(torch.tensor(sigma2, dtype=torch.float64))
k = kernel.KIMQ(s2=sigma2)

In [187]:
k.eval(X, Y)

tensor([[0.7697453516, 0.7727955921],
        [0.7610091801, 0.6633383865],
        [0.8092496917, 0.6952153393]], dtype=torch.float64,
       grad_fn=<PowBackward0>)

In [188]:
torch.abs(k.eval(X, Y)-k_.eval(X, Y)).max()

tensor(1.1102230246e-16, dtype=torch.float64, grad_fn=<MaxBackward1>)

In [189]:
torch.abs(k_.gradXY_sum(X, Y) - k.gradXY_sum(X, Y)).max()

tensor(2.7755575616e-17, dtype=torch.float64, grad_fn=<MaxBackward1>)

In [190]:
torch.abs(k_.gradX(X, Y) - k.gradX(X, Y)).max()

tensor(2.7755575616e-17, dtype=torch.float64, grad_fn=<MaxBackward1>)